In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [21]:
def create_date_features(df):
    df['month'] = df['date'].dt.month
    df['day_of_month'] = df['date'].dt.day
    df['day_of_year'] = df['date'].dt.dayofyear
    df['week_of_year'] = df['date'].dt.isocalendar().week
    df['day_of_week'] = df['date'].dt.dayofweek
    df['year'] = df['date'].dt.year
    return df

In [27]:
oil = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
sample_submission = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv")
holidays_events = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

In [28]:
train['date'] = pd.to_datetime(train['date'])
train['sales'] = np.log1p(train['sales'])

test['date'] = pd.to_datetime(test['date'])

In [29]:
df = pd.concat([train, test], axis=0)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3029400 entries, 0 to 28511
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 161.8+ MB


In [31]:
df = create_date_features(df)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3029400 entries, 0 to 28511
Data columns (total 12 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            int64         
 1   date          datetime64[ns]
 2   store_nbr     int64         
 3   family        int64         
 4   sales         float64       
 5   onpromotion   int64         
 6   month         int32         
 7   day_of_month  int32         
 8   day_of_year   int32         
 9   week_of_year  UInt32        
 10  day_of_week   int32         
 11  year          int32         
dtypes: UInt32(1), datetime64[ns](1), float64(1), int32(5), int64(4)
memory usage: 234.0 MB


In [32]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['family'] = label_encoder.fit_transform(df['family'])

In [34]:
df = df[df['date'] > '2013-12-31']
x = df[df['date'] <= '2017-08-15'].drop('date', axis=1)
test_x = df[df['date'] > '2017-08-15'].drop('date', axis=1)

In [35]:
y = x['sales']
x = x.drop(['sales'], axis=1)
test_x = test_x.drop(['sales'], axis=1)

In [36]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def train_linear_regressor(X, y):
    # 학습 및 테스트 데이터로 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 선형 회귀 모델 생성 및 학습
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # 학습된 모델 평가
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    # 평가 지표 계산
    train_rmse = mean_squared_error(y_train, train_pred, squared=False)
    test_rmse = mean_squared_error(y_test, test_pred, squared=False)
    
    print("Train RMSE:", train_rmse)
    print("Test RMSE:", test_rmse)
    
    return model

# X는 입력 특성(feature), y는 목표 변수(target)입니다.
# X와 y는 각각 특성 행렬(feature matrix)과 타겟 벡터(target vector)여야 합니다.
# 이 예제에서는 X_train과 y_train이 주어졌다고 가정합니다.
# X_train, y_train은 모델을 학습하기 위한 훈련 데이터입니다.
# X_test, y_test는 모델을 평가하기 위한 테스트 데이터입니다.

# 모델 훈련 예시:
model = train_linear_regressor(x, y)

# 모델로 예측하는 예시:
predictions = model.predict(test_x)

Train RMSE: 2.477334399741697
Test RMSE: 2.4752552404922383


In [37]:
model.fit(x,y)
pred_y = model.predict(test_x)
df = pd.DataFrame(pred_y,columns=['sales'])
df.to_csv(f"{model}.csv",index_label='id')

In [38]:
from IPython.display import FileLink

def save_and_download_csv_with_custom_index(df, filename, start_index=3000888):
    # 인덱스를 조정합니다.
    df.index = range(start_index, start_index + len(df))
    # DataFrame을 CSV 파일로 저장합니다.
    df.to_csv(filename)
    # 파일을 다운로드할 수 있는 HTML 링크를 생성하고 표시합니다.
    display(FileLink(filename))

# df[0][0] = 'id'
# 예제 DataFrame인 df를 사용하여 파일을 저장하고 다운로드하는 예시:
save_and_download_csv_with_custom_index(df, "submission.csv")


/kaggle/working/submission.csv

# ----------------------------------------------------------------------------------------

In [ ]:
# 학습 데이터(train_x)와 학습 데이터의 정답(train_y)으로 구분
train_y = train['sales']
train.drop(train.columns[4], axis=1, inplace=True)
train_x = train
test_x = test

In [ ]:
# LabelEncoder를 쓸려는데
# train 데이터에 없는 값이 test 데이터 있는 걸 방지 하기 위해 둘이 합침
combined_dates = pd.concat([train_x['date'], test_x['date']], axis=0)
combined_family = pd.concat([train_x['family'], test_x['family']], axis=0)

In [ ]:
# 숫자 문자 섞여있음 (다 문자로 바꿈 -> LabelEncoder 쓰기 위해)
combined_dates = combined_dates.astype(str)
combined_family = combined_family.astype(str)
train_x['date'] = train_x['date'].astype(str)
test_x['date'] = test_x['date'].astype(str)
train_x['family'] = train_x['family'].astype(str)
test_x['family'] = test_x['family'].astype(str)

In [ ]:
# 문자를 숫자로 바꿈
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
combined_dates = label_encoder.fit_transform(combined_dates)
train_x['date'] = label_encoder.transform(train_x['date'])
test_x['date'] = label_encoder.transform(test_x['date'])
combined_family = label_encoder.fit_transform(combined_family)
train_x['family'] = label_encoder.transform(train_x['family'])
test_x['family'] = label_encoder.transform(test_x['family'])

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [ ]:
# 여러 회귀 모델로 MSE 값 찾기
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, RandomForestRegressor, VotingRegressor
from sklearn.model_selection import cross_val_score
models = [
    ExtraTreesRegressor(),
    GradientBoostingRegressor(),
    HistGradientBoostingRegressor(),
    RandomForestRegressor()
]
for model in models:
  model.fit(train_x,train_y)
  pred_y = model.predict(test_x)
  mse = mean_squared_error(test_y, pred_y)
  print(f'{model}: %f' %mse)